In [ ]:
#default_exp transforms

In [ ]:
#export
from mantisshrimp.imports import *
from mantisshrimp.core import *
from mantisshrimp.data.all import *

# Transform
> Trasformers

In [ ]:
#export
class Transformer:
    def __init__(self, tfms): self.tfms=tfms
    def __call__(self, record): return self.apply(record)

In [ ]:
#export
class EmptyTransformer(Transformer):
    def __init__(self, tfms=None): super().__init__(tfms=tfms)
    def apply(self, record):
        im = open_img(record.iinfo.fp)
        return im, record

In [ ]:
#export
class AlbumentationTransformer(Transformer):
    def __init__(self, tfms):
        self.bbox_params=A.BboxParams(format='pascal_voc', label_fields=['oids'])
        super().__init__(tfms=A.Compose(tfms, bbox_params=self.bbox_params))
    
    def apply(self, record):
        d = self.tfms(
            image = open_img(record.iinfo.fp),
            bboxes = [o.xyxy for o in record.annot.bboxes],
            masks = Mask.from_segs(record.annot.segs, record.iinfo.h, record.iinfo.w).data,
            oids = record.annot.oids,
            keypoints = record.annot.kpts,
        )
        # TODO: Don't use dicts
        h,w,_ = d['image'].shape
        iinfo = dict(h=h, w=w)
        annot = dict(
            oids=d['oids'],
            bboxes=[BBox.from_xyxy(*o) for o in d['bboxes']] if notnone(d['bboxes']) else None,
            segs=Mask(np.stack(d['masks'])) if notnone(d['masks']) else None,
        #     kpts=res['keypoints'] # TODO
        )
        return d['image'], record.new(iinfo=iinfo, annot=annot)

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 02_data.core.ipynb.
Converted 04_data.annotations.ipynb.
Converted 06_transforms.ipynb.
Converted 07_data.load.ipynb.
Converted 08_models.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted index.ipynb.
